In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import time
from time import sleep
from datetime import datetime
import re

import numpy as np
import pandas as pd

import smtplib
from email.mime.text import MIMEText
from email import encoders, utils
from email.mime.multipart import MIMEMultipart

In [2]:
keywords = ["邊坡", "崩塌", "復建"]
orgkws = ["水土保持局臺北分局", "羅東林區管理處 ", "第四區養護工程處", "宜蘭縣"]

In [3]:
def get_tender_by_kw(kws, loc):
    driver = webdriver.Chrome()
    tender_list = []
    for k in kws:
        bidSystemUrl = "https://web.pcc.gov.tw/pis/"
        browse = driver.get(bidSystemUrl)
        homepage = driver.page_source
        query = driver.find_element_by_name("tenderName")
        query.clear()
        query.send_keys(k)
        queryCate = driver.find_element_by_css_selector("#basicRadProctrgCate1")
        queryCate.click()
        queryTime = driver.find_element_by_css_selector("#basicIsSpdtDateTypeId")
        queryTime.click()
        query_loc = driver.find_element_by_name("orgName")
        query_loc.send_keys(loc)
        driver.execute_script('return basicTenderSearch()')
        sleep(1) #等頁面回傳
        bidpage = driver.page_source
        soup = BeautifulSoup(bidpage, 'html.parser')
        tender_table = soup.find_all("tbody")[-1].find_all("tr")
        if (len(tender_table) == 1) and (tender_table[0].text == "無符合條件資料"):
            continue
        else:
            tender_list += tender_table
    driver.close()
    return list(set(tender_list))

def get_tender_by_org(orgs):
    driver = webdriver.Chrome()
    tender_list = []
    for o in orgs:
        bidSystemUrl = "https://web.pcc.gov.tw/pis/"
        browse = driver.get(bidSystemUrl)
        homepage = driver.page_source
        query = driver.find_element_by_name("orgName")
        query.clear()
        query.send_keys(o)
        queryCate = driver.find_element_by_css_selector("#basicRadProctrgCate1")
        queryCate.click()
        queryTime = driver.find_element_by_css_selector("#basicIsSpdtDateTypeId")
        queryTime.click()
        driver.execute_script('return basicTenderSearch()')
        sleep(1) #等頁面回傳
        bidpage = driver.page_source
        soup = BeautifulSoup(bidpage, 'html.parser')
        tender_table = soup.find_all("tbody")[-1].find_all("tr")
        if (len(tender_table) == 1) and (tender_table[0].text == "無符合條件資料"):
            continue
        else:
            tender_list += tender_table
    driver.close()
    return list(set(tender_list))

def get_tender_info(tender):
    return {
        "機關名稱": "".join(tender.find_all("td")[1].text.split()), 
        "標案案號&標案名稱": " ".join(tender.find_all("td")[2].text.split()), 
        "傳輸次數": "".join(tender.find_all("td")[3].text.split()), 
        "招標方式": "".join(tender.find_all("td")[4].text.split()), 
        "採購性質": "".join(tender.find_all("td")[5].text.split()), 
        "公告日期": "".join(tender.find_all("td")[6].text.split()), 
        "截止投標": "".join(tender.find_all("td")[7].text.split()), 
        "預算金額": "".join(tender.find_all("td")[8].text.split()), 
        "url": tender.find_all("td")[2].find('a').get('href').replace("/prkms/urlSelector/common/tpam?pk", "https://web.pcc.gov.tw/tps/QueryTender/query/searchTenderDetail?pkPmsMain"),
#         "url": tender.find_all("td")[2].find('a').get('href')
           }

In [3]:
driver = webdriver.Chrome()
bidSystemUrl = "https://web.pcc.gov.tw/pis/"
browse = driver.get(bidSystemUrl)

In [4]:
# query = driver.find_element_by_name("tenderName")
# query.clear()
# query.send_keys("吃飯")

query = driver.find_element_by_name("orgName")
query.clear()
query.send_keys("宜蘭")

In [5]:
queryCate = driver.find_element_by_css_selector("#basicRadProctrgCate1")
queryCate.click()
queryTime = driver.find_element_by_css_selector("#basicIsSpdtDateTypeId")
queryTime.click()

In [6]:
driver.execute_script('return basicTenderSearch()')
sleep(1) #等頁面回傳
bidpage = driver.page_source

In [7]:
soup = BeautifulSoup(bidpage, 'html.parser')

In [8]:
tender_table = soup.find_all("tbody")[-1].find_all("tr")

In [9]:
tender_table

[<tr class="tb_b2">
 <td class="no_blank" style="text-align:center;">
 				1
 			</td>
 <td>
 				宜蘭縣政府
 			</td>
 <td>
 				AGCO111022
 				<br/>
 <a href="/prkms/urlSelector/common/tpam?pk=NzAwMDE0ODY=">
 <u> <span id="1">中華社區粗坑路排水改善工程</span></u>
 </a>
 </td>
 <td>
 <a href="/prkms/urlSelector/common/tpam?pk=NzAwMDE0ODY=">
 <u>01</u>
 </a>
 </td>
 <td>
 				  
 	 			公開招標 
 			</td>
 <td class="no_blank">
 				
 					  
 		 			工程類
 	 			
 			</td>
 <td class="no_blank" style="text-align:center;">
 				
 				111/06/08
 
 			</td>
 <td class="no_blank" style="text-align:center;">
 				
 				111/06/20
 
 			</td>
 <td class="no_blank">
 <span class="no_blank R"> 
 						
 						
 							
 								
 									5,767,670
 								
 							
 						
 					
 				</span>
 </td>
 <td>
 <div class="bt_cen1 no_blank">
 <a href="/prkms/urlSelector/common/tpam?pk=NzAwMDE0ODY="> 檢視 </a>
 </div>
 </td></tr>,
 <tr class="tb_b3">
 <td class="no_blank" style="text-align:center;">
 				2
 			</td>
 <td>
 	

In [12]:
get_tender_info(tender_table[1])

{'機關名稱': '宜蘭縣羅東鎮成功國民小學',
 '標案案號&標案名稱': 'cges1110607 109年度千禧樓老舊廁所整修工程',
 '傳輸次數': '02',
 '招標方式': '公開招標',
 '採購性質': '工程類',
 '公告日期': '111/06/08',
 '截止投標': '111/06/13',
 '預算金額': '3,056,471',
 'url': 'https://web.pcc.gov.tw/tps/QueryTender/query/searchTenderDetail?pkPmsMain=NzAwMDE0NzM='}

In [4]:
t_ks = []
for loc in ["臺北", "新北", "桃園", "新竹", "宜蘭"]:
    t = get_tender_by_kw(keywords, loc)
    t_ks += t
t_orgs = get_tender_by_org(orgkws)
t_list = list(set(t_ks+t_orgs))

In [5]:
len(t_ks), len(t_orgs), len(t_list)

(8, 54, 62)

In [6]:
get_tender_info(t_list[0])

{'機關名稱': '交通部公路總局第四區養護工程處',
 '標案案號&標案名稱': '1114B036 台7丁線5k+500~6k+500路面改善工程',
 '傳輸次數': '01',
 '招標方式': '公開招標',
 '採購性質': '工程類',
 '公告日期': '111/05/31',
 '截止投標': '111/06/13',
 '預算金額': '9,963,836',
 'url': 'https://web.pcc.gov.tw/tps/QueryTender/query/searchTenderDetail?pkPmsMain=NTM4MTk1MDU='}

In [7]:
now = datetime.now().strftime("%Y/%m/%d")
text = f"<h>{now}等標期內標案</h>"

In [8]:
tender_df = pd.DataFrame()
for t in t_list:
    temp_df = pd.DataFrame()
    t_info = get_tender_info(t)
    for k, v in t_info.items():
        temp_df[k] = [v]
    tender_df = pd.concat([tender_df, temp_df], ignore_index=True)
tender_df.sort_values("截止投標", inplace=True, ignore_index=True)
tender_df.drop_duplicates(keep='last', ignore_index=True, inplace=True)

In [9]:
tender_df['url'] = '<a href=' + tender_df['url'] + '><div>連結</div></a>'
text += tender_df.to_html(escape=False).replace("\n", "")

In [10]:
tender_df

,機關名稱,標案案號&標案名稱,傳輸次數,招標方式,採購性質,公告日期,截止投標,預算金額,url
0,行政院農業委員會水土保持局臺北分局,111-LL-07-1-002 雪霧鬧地區坡地排水加強改善工程,03,公開招標,工程類,111/05/27,111/06/08,"4,408,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
1,宜蘭縣立南澳高級中學,1110602 111年圖書館二館空間活化改善工程採購案,01,公開取得報價單或企劃書,工程類,111/06/02,111/06/08,"988,672",<a href=https://web.pcc.gov.tw/tps/QueryTender...
2,行政院農業委員會水土保持局臺北分局,111FWS3001-003 卡拉溪野溪囚砂設施治理工程第一期,03,公開招標,工程類,111/05/27,111/06/08,"17,913,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
3,行政院農業委員會水土保持局臺北分局,111FWS3001-006 石門-0044崩塌地處理工程,03,公開招標,工程類,111/05/26,111/06/08,"8,842,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
4,宜蘭縣政府,WRCO111036 宜蘭縣仁山水土保持戶外教室優質化四期工程,01,公開招標,工程類,111/05/30,111/06/09,"3,334,983",<a href=https://web.pcc.gov.tw/tps/QueryTender...
5,宜蘭縣政府,AGCO111016 111年度南澳鄉農宜南001暨朝陽農路設施改善工程,01,公開招標,工程類,111/05/30,111/06/09,"2,886,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
6,宜蘭縣蘇澳鎮永樂國民小學,YL202201 (更正公告) 111年度教育部國民及學前教育署補助宜蘭縣蘇澳鎮永樂國小校舍...,01,公開取得報價單或企劃書,工程類,111/06/08,111/06/13,"980,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
7,行政院農業委員會水土保持局臺北分局,111RA-01-001 員山鄉結頭份社區花甲療育園區營造工程,02,公開招標,工程類,111/06/08,111/06/13,"8,061,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...
8,交通部公路總局第四區養護工程處,1114B038 獨立山工務段四季監工站111年度預約經常性災害搶修工程(第2次),01,公開招標,工程類,111/06/02,111/06/13,"47,442,839",<a href=https://web.pcc.gov.tw/tps/QueryTender...
9,行政院農業委員會水土保持局臺北分局,111-FRAP-03-1-107 下灣子坑溝整治工程,03,公開招標,工程類,111/06/01,111/06/13,"3,549,000",<a href=https://web.pcc.gov.tw/tps/QueryTender...


In [11]:
mailList = ['chongjing3370@gmail.com', 'tusty9292@gmail.com', 'fish892555@gmail.com']

In [12]:
me = 'tusty9292@gmail.com'
mail = MIMEMultipart()
mail['Subject'] = "政府採購公告網 "+str(time.strftime("%Y/%m/%d-%H%M"))
mail['From'] = 'chongjing3370@gmail.com'
mail['Date'] = utils.formatdate(localtime = 1)
mail['Message-ID'] = utils.make_msgid()
mail['Content-Type'] = "text/calendar; charset=utf-8"
body = MIMEText(text, 'html', "utf-8")
body.set_charset("utf-8")
mail.attach(body)

In [13]:
with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
    try:
        smtp.ehlo()  # 驗證SMTP伺服器
        smtp.starttls()  # 建立加密傳輸
        smtp.login('chongjing3370@gmail.com', 'njhxfuhvwdmdjgds')  # 登入寄件者gmail
        smtp.sendmail(me, mailList, mail.as_string())  # 寄送郵件
        print("Complete!")
    except Exception as e:
        print("Error message: ", e)

Complete!
